In [ ]:
!pip3 install --upgrade --user google-cloud-aiplatform
!pip install yfinance

In [ ]:
import vertexai
from vertexai.preview.generative_models import GenerativeModel, Image


PROJECT_ID = "cryptic-skyline-411516"
REGION = "us-central1"
vertexai.init(project=PROJECT_ID, location=REGION)

model = GenerativeModel("gemini-1.5-flash-002")
request = "What is the latest Microsoft Stock price and tell me what date you have the price from?"
response = model.generate_content(request)
print(response.text)

In [ ]:
import yfinance as yf
from vertexai.generative_models import (
    Content,
    FunctionDeclaration,
    GenerationConfig,
    GenerativeModel,
    Part,
    Tool,
    ToolConfig
)

In [ ]:
# Function to fetch stock prices
def get_stock_price(ticker_symbol: str) -> dict:
    """
    Fetches the latest stock price for a given ticker symbol.

    Args:
        ticker_symbol (str): The stock's ticker symbol (e.g., "AAPL").

    Returns:
        dict: A dictionary containing the ticker symbol and latest price.
    """
    try:
        ticker = yf.Ticker(ticker_symbol)
        stock_data = ticker.history(period="1d")
        if not stock_data.empty:
            latest_price = stock_data['Close'].iloc[-1]
            return {"ticker": ticker_symbol, "price": latest_price}
        else:
            return {"error": f"No data available for ticker {ticker_symbol}."}
    except Exception as e:
        return {"error": str(e)}

In [ ]:


function_name = "get_stock_price"
# Define the tool schema
stock_price_tool = FunctionDeclaration(
    name=function_name,
    description="Fetches the latest stock price for a given ticker symbol.",
    parameters={
        "type": "object",
        "properties": {
            "ticker_symbol": {
                "type": "string",
                "description": "The stock ticker symbol, e.g., AAPL, TSLA.",
            }
        },
        "required": ["ticker_symbol"]
    }
)
# Define a tool that includes some of the functions that we declared earlier
tool = Tool(
    function_declarations=[stock_price_tool],
)

response = model.generate_content(request, 
                                  tools=[tool],
                                  tool_config=ToolConfig(
                                    function_calling_config=ToolConfig.FunctionCallingConfig(
                                        # ANY mode forces the model to predict only function calls
                                        mode=ToolConfig.FunctionCallingConfig.Mode.ANY,
                                        # Allowed function calls to predict when the mode is ANY. If empty, any  of
                                        # the provided function calls will be predicted.
                                        allowed_function_names=["get_stock_price"],
                                    )
                                ))
function_call = response.candidates[0].function_calls[0]
user_prompt_content = Content(
    role="user",
    parts=[
        Part.from_text(request),
    ],
)
# Check the function name that the model responded with, and make an API call to an external system
if function_call.name == function_name:
    # Extract the arguments to use in your API call
    ticker = function_call.args["ticker_symbol"]  # noqa: F841
    print("LLM  response (Ticker Symnp;) : ", ticker)
    print("Use the LLM Response now to call the LLM Function...")
    # Here you can use your preferred method to make an API request to fetch the current weather, for example:
    api_response = get_stock_price(ticker)

    # In this example, we'll use synthetic data to simulate a response payload from an external API

# Return the API response to Gemini so it can generate a model response or request another function call
response = model.generate_content(
    [
        user_prompt_content,  # User prompt
        response.candidates[0].content,  # Function call response
        Content(
            parts=[
                Part.from_function_response(
                    name=function_name,
                    response={
                        "content": api_response,  # Return the API response to Gemini
                    },
                ),
            ],
        ),
    ],
    tools=[tool],
)

# Get the model response
print("LLM Function response : ", response.text)